In [9]:
import random
from sympy import isprime, nextprime
from math import gcd

Generate a random prime number of given approx `bits` length

In [10]:
def genPrime(bits):
    while True:
        num = random.getrandbits(bits)
        num |= (1 << bits - 1) | 1  # Ensure high bit is set and it's odd
        if isprime(num):
            return num

Compute modular inverse using Extended Euclidean Algorithm

In [11]:
def modinv(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        a, m = m, a % m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

## Generate RSA key pair

In [12]:
def genKeyPairs(keysize):
    half = keysize // 2
    p = genPrime(half)
    q = genPrime(half)
    while p == q:
        q = genPrime(half)

    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537  # Standard public exponent

    if gcd(e, phi) != 1:
        # Retry till valid pair
        return genKeyPairs(keysize)

    d = modinv(e, phi)

    return (e, n), (d, n)

# Encryption

In [13]:
def encrypt(public_key, plaintext):
    e, n = public_key
    # Convert string to int then encrypt
    plaintextInt = int.from_bytes(plaintext.encode(), byteorder='big')
    cipherTextInt = pow(plaintextInt, e, n)
    return cipherTextInt

# Decryption

In [14]:
def decrypt(private_key, ciphertext_int):
    d, n = private_key
    decryptedInt = pow(ciphertext_int, d, n)
    decryptedBytes = decryptedInt.to_bytes((decryptedInt.bit_length() + 7) // 8, byteorder='big')
    return decryptedBytes


# Test Phase

In [15]:
keySize = 6

In [16]:
publicKey, privateKey = genKeyPairs(keySize)

## Public Key

In [17]:
publicKey

(65537, 35)

## Private Key

In [18]:
privateKey

(17, 35)

In [19]:
message = "Msg by Vaibhav Pandey"

In [20]:
encrypted = encrypt(publicKey, message)
encrypted

29

In [21]:
decrypted = decrypt(privateKey, encrypted)
decrypted

b'\x1d'